In [ ]:
# 处理照片Exif信息

import exifread
import os
import sys
import shutil

# 读取EXIF信息
def getPhotoExif(filename):
    fd = open(filename,'rb')
    tags = exifread.process_file(fd)
    fd.close()
    return(tags)

def movePhoto(path,dst):
    n = 1
    m = 0
    for root, dirs, files in os.walk(path):
        for filename in files:
            filename = os.path.join(root,filename)
            f,ext = os.path.splitext(filename)
            if ext.lower() not in ('.jpg','.png','.mp4','.gif'):
                continue
            tags = getPhotoExif(filename)
            #print("----------------------------------------------------------------")            
            try:
                date = str(tags['EXIF DateTimeOriginal']).replace(":","-")[:10]
                #print(n,"文件:",filename,"相机:",tags['Image Model'],"，拍摄时间：",date)
                pwd = root + "\\" + date
                year = date[0:4]
                yearpath = dst+"\\"+year

                if not os.path.exists(yearpath):
                    os.mkdir(yearpath)
                daypath = yearpath+"\\"+date

                if not os.path.exists(daypath):
                    os.mkdir(daypath)
                shutil.move(filename,daypath)
                print(n,filename+"  ----->  "+daypath)
                n = n + 1
            except:
                print("照片",filename,"没有EXIF数据")
                m = m + 1
                pass
    
    print("共移动" ,n-1,"文件，",m,"个文件未移动")

def main():
    
    msg = """使用方法：
        python movePhoto.py 源文件夹 目标文件夹
        """
    
    if len(sys.argv) < 3:
        print(msg)
    else:
        movePhoto(sys.argv[1],sys.argv[2])
    
if __name__ == '__main__':
    
    main()

In [ ]:
import os
import re
import requests

word = input("请输入要搜索的图片：")
if not os.path.exists('c:/Temp/' + word):
    os.mkdir('c:/Temp/' + word)

url = "https://image.baidu.com/search/index?tn=baiduimage&ipn=r&ct=201326592\
&cl=2&lm=-1&st=-1&fm=result&fr=&sf=1&fmq=1540822430688_R&pv=&ic=0&nc=1&z=&se=1\
&showtab=0&fb=0&width=&height=&face=0&istype=2&ie=utf-8&word=" + word

header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:63.0) \
Gecko/20100101 Firefox/63.0"}

r = requests.get(url, headers=header, timeout=3).content.decode("utf-8")
# r.encoding = "utf-8"
# r = r.text

img_list = re.findall('"objURL":"(.*?)",', r)
# print(img_list)

for img in img_list:
    print(img)
    end = re.search('(.jpg|.png|.gif|.jpeg)$', img)
    if end == None:
        img = img +'.jpg'

    path = re.sub('\?|\/', '', img[-10:])
    # print(path)
    try:
        with open('c:/Temp/' + word +'/{}'.format(path), 'ab') as f:
            ret = requests.get(img, headers=header, timeout=3)
            f.write(ret.content)
    except Exception:
        pass


In [2]:
from selenium import webdriver
import re
import time

browser = webdriver.Chrome()
browser.get('https://www.csdn.net/')
browser.implicitly_wait(10)

i = 0
for i in range(5):###设置下拉5次，如果想获取更多信息，增加下拉次数即可
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')##下拉，execute_script可以将进度条下拉到最底部
    time.sleep(5)##睡眠一下，防止网络延迟，卡顿等

data = []
pattern = re.compile('<li.*?blog".*?>.*?title">.*?<a.*?>(.*?)</a>.*?<dd.*?name">.*?<a.*?blank">(.*?)</a>'
                     '.*?<span.*?num">(.*?)</span>.*?text">(.*?)</span>.*?</li>',re.S)
items = re.findall(pattern,browser.page_source)##这里网页源代码为下拉5次后的代码
for item in items:
    data.append({
        'Title':item[0].strip(),
        'Author' : item[1].strip(),
        'ReadNum' : item[2] + item[3]
    })
print(data)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
from _md5 import md5
import requests
from selenium import webdriver
import re
from multiprocessing import Pool

# 获取完整网页源码 
def get_one_html(url): 
    browser = webdriver.Phantomjs()
    browser.get(url) 
    p_source = browser.page_source
    browser.close() 
    return p_source 

# 用正则表达式提取链接并且返回URL 
def parse_html(html): 
    pattern = re.compile('<li.*?<img src="(.*?)".*?/>.*?</li>', re.S)
    urls = re.findall(pattern, html)
    print(urls)
    return urls

# 下载图片，注意的是文件写入的是二进制，并且文件名使用MD5生成固定长度的十六进制，防止文件名冲突，index表示页数 
def download_imgs(urls):
    print('one page')
    for index, url in enumerate(urls): 
        response = requests.get(url) 
        content = response.content
        file_path = '{0}\\{1}{2}{3}'.format('E:\\jdpics', md5(content).hexdigest(), index, '.jpg')
        with open(file_path, 'wb') as f: 
            f.write(content) 
            print('当前页面爬取完成')

# 构造一个可以变化的URL，并下载
def main(i): 
    url = 'http://jandan.net/ooxx/page-'+str(i)+'#comments' 
    html = get_one_html(url) 
    urls = parse_html(html)
    download_imgs(urls) 
    
# 实现一个多进程爬虫 
    
if __name__ == '__main__':
    # for i in range(1, 85):
    #     url = 'http://jandan.net/ooxx/page-'+str(i)+'#comments'
    #     main(url)
    pool = Pool()
    pool.map(main, [i for i in range(1, 85)])


In [1]:
import os
import re
import requests
from selenium import webdriver
import time

word = "毛泽东"

url = "https://image.baidu.com/search/index?tn=baiduimage&ipn=r&ct=201326592\
&cl=2&lm=-1&st=-1&fm=result&fr=&sf=1&fmq=1540822430688_R&pv=&ic=0&nc=1&z=&se=1\
&showtab=0&fb=0&width=&height=&face=0&istype=2&ie=utf-8&word=" + word

header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:63.0) \
Gecko/20100101 Firefox/63.0"}

browser = webdriver.Firefox()
browser.get(url)
browser.implicitly_wait(10)

i = 0
for i in range(5):# 设置下拉5次
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')# 下拉到最底部
    time.sleep(5)# 防止网络延迟，卡顿等

r = browser.page_source
browser.close()
print(r)

img_list = re.findall('"objURL":"(.*?)",', r)
print(img_list)

for img in img_list:
#    print(img)

'''
    path = re.sub('\?|\/', '', img[-10:])

    try:
        with open('c:/Temp/' + word +'/{}'.format(path), 'ab') as f:
            ret = requests.get(img, headers=header, timeout=30)
            f.write(ret.content)
    except Exception:
        pass
'''

IndentationError: expected an indented block (<ipython-input-1-7ae6bd9e8918>, line 41)

In [ ]:
import requests
import re

class TaobaoSpider(object):

    def __init__(self):
        self.url = "https://rate.tmall.com/list_detail_rate.htm?itemId=559979448696\
&spuId=349188091&sellerId=2646031546&order=3&currentPage="
        self.page = 1
        self.headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:63.0) \
Gecko/20100101 Firefox/63.0"}
        
    def start_request(self):
        for i in range(10):
            print("正在爬取第%d页...--------------------------------------------------------------------------" % self.page)
            content = requests.get(url=self.url + str(self.page), headers=self.headers).content.decode()
            self.content_re(content)
            self.page += 1
    
    def content_re(self, content):
        ratedate = re.findall(r'"rateDate":"(.*?)",',content)
        ratecontent = re.findall(r'"rateContent":"(.*?)",', content)
        for rd, rc in zip(ratedate,ratecontent):
            print("【"+rd+"】", rc)
        
if __name__ == "__main__":
    spider = TaobaoSpider()
    spider.start_request()

In [ ]:
import requests
import re
from lxml import etree

class ZbSpider(object):
    def __init__(self):
        self.url = "http://www.ccgp-qinghai.gov.cn/jilin/zbxxController.form?"
        self.headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:63.0) \
Gecko/20100101 Firefox/63.0"}
        self.page = 0
        
    def start_request(self):
        lx = input("请输入公告类型（1.省级 2.州市县级）：")
        pg = input("请输入爬取页数：")
        for self.page in range(int(pg)):
            print("正在爬取中标公告第【%d】页...-----------------------------" % self.page)
            html = requests.get(self.url+"declarationType=W"+"&type="+lx+"&pageNo="+
str(self.page), headers=self.headers).content.decode()
            self.zbxx(html)
        
        self.page = 0
        for self.page in range(int(pg)):
            print("正在爬取废流标公告第【%d】页...---------------------------" % self.page)
            html = requests.get(self.url+"declarationType=F"+"&type="+lx+"&pageNo="+
str(self.page), headers=self.headers).content.decode()
            self.zbxx(html)

    #中标公告信息获取
    
    def zbxx(self, html):
        #stitle = re.findall(r'title="(.*?)"', html)    #公告标题
        surl = re.findall(r'" href="(.*?)">', html)
        for su in surl:
            #实际公告页面在框架内，去除框架页多余字符，合成实际url
            sggurl = su.replace('ftl/jilin/noticeDetail.jsp?htmlURL=','')
            print(sggurl)
            sgg = requests.get(sggurl, headers=self.headers).content
            s = etree.HTML(sgg)
            name = s.xpath('/html/body/div/table/tbody/tr[2]/td[2]/p/span/text()')
            date = s.xpath('/html/body/div/table/tbody/tr[8]/td[2]/p/span/text()')
            man = s.xpath('/html/body/div/table/tbody/tr[13]/td[2]/p/span/text()')
            print('【'+date[0]+'】',name[0],'【'+man[0]+'】')
    
if __name__ == "__main__":
    zb = ZbSpider()
    zb.start_request()

In [ ]:
#lx = input("请输入公告类型（1.省级 2.州市县级）：")
pg = input("请输入爬取页数：")
#print(type(lx))
print(type(int(pg)))
#n=
for i in range(int(pg)):
    print(i)

In [ ]:
import requests
from lxml import etree
url = "http://www.ccgp-qinghai.gov.cn/html/2018/11/8/W062700.html"

header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:63.0) \
Gecko/20100101 Firefox/63.0"}

r = requests.get(url, headers=header).content#.decode("gb2312")
s = etree.HTML(r)
name = s.xpath('/html/body/div/table/tbody/tr[2]/td[2]/p/span/text()')
date = s.xpath('/html/body/div/table/tbody/tr[8]/td[2]/p/span/text()')
man = s.xpath('/html/body/div/table/tbody/tr[13]/td[2]/p/span/text()')
print(name[0],date[0],man[0])